Learn GEE

https://developers.google.com/earth-engine/guides/image_visualization

In [1]:
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    EE_PROJECT_ID = userdata.get('EE_PROJECT_ID') 
else:
    from dotenv import load_dotenv
    import os
    load_dotenv()  # take environment variables
    EE_PROJECT_ID = os.getenv('EE_PROJECT_ID')
    
# Set up GEE API
import ee
ee.Authenticate()
ee.Initialize(project=EE_PROJECT_ID)

# Import other libs
import geemap


## Masking

In [3]:
# Sentinel-2 image collection for July 2021 intersecting a point of interest.
# Reflectance, cloud probability, and scene classification bands are selected.
col = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2021-07-01', '2021-08-01')
    .filterBounds(ee.Geometry.Point(-122.373, 37.448))
    .select('B.*|MSK_CLDPRB|SCL')
)

# Visualization parameters for reflectance RGB.
vis_refl = {'bands': ['B11', 'B8', 'B3'], 'min': 0, 'max': 4000}
m = geemap.Map()
m.set_center(-122.373, 37.448, 9)
m.add_layer(col, vis_refl, 'Collection reference', False)

# Reduce the collection to a single image using a variety of methods.
mean = col.mean()
m.add_layer(mean, vis_refl, 'Mean (B11, B8, B3)')

median = col.median()
m.add_layer(median, vis_refl, 'Median (B11, B8, B3)')

min = col.min()
m.add_layer(min, vis_refl, 'Min (B11, B8, B3)')

max = col.max()
m.add_layer(max, vis_refl, 'Max (B11, B8, B3)')

sum = col.sum()
m.add_layer(
    sum, {'bands': ['MSK_CLDPRB'], 'min': 0, 'max': 500}, 'Sum (MSK_CLDPRB)'
)

product = col.product()
m.add_layer(
    product,
    {'bands': ['MSK_CLDPRB'], 'min': 0, 'max': 1e10},
    'Product (MSK_CLDPRB)',
)

# ee.ImageCollection.mode returns the most common value. If multiple mode
# values occur, the minimum mode value is returned.
mode = col.mode()
m.add_layer(
    mode, {'bands': ['SCL'], 'min': 1, 'max': 11}, 'Mode (pixel class)'
)

# ee.ImageCollection.count returns the frequency of valid observations. Here,
# image pixels are masked based on cloud probability to add valid observation
# variability to the collection. Note that pixels with no valid observations
# are masked out of the returned image.
not_cloud_col = col.map(
    lambda img: img.updateMask(img.select('MSK_CLDPRB').lte(10))
)
count = not_cloud_col.count()
m.add_layer(count, {'min': 1, 'max': 5}, 'Count (not cloud observations)')

# ee.ImageCollection.mosaic composites images according to their position in
# the collection (priority is last to first) and pixel mask status, where
# invalid (mask value 0) pixels are filled by preceding valid (mask value >0)
# pixels.
mosaic = not_cloud_col.mosaic()
m.add_layer(mosaic, vis_refl, 'Mosaic (B11, B8, B3)')
m

Map(center=[37.448, -122.373], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…

In [6]:
# Load three NAIP quarter quads in the same location, different times.
naip_2004_2012 = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(ee.Geometry.Point(-71.08841, 42.39823))
    .filterDate('2004-07-01', '2012-12-31')
    .select(['R', 'G', 'B'])
)

# Temporally composite the images with a maximum value function.
composite = naip_2004_2012.max()

m = geemap.Map()
m.set_center(-71.12532, 42.3712, 12)
m.add_layer(composite, {}, 'max value composite')
m

Map(center=[42.3712, -71.12532], controls=(WidgetControl(options=['position', 'transparent_bg'], position='top…

In [ ]:
# Load four 2012 NAIP quarter quads, different locations.
naip_2012 = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(
        ee.Geometry.Rectangle(-71.17965, 42.35125, -71.08824, 42.40584)
    )
    .filterDate('2012-01-01', '2012-12-31')
) # 4 images

# Spatially mosaic the images in the collection and display.
mosaic = naip_2012.mosaic()
m = geemap.Map()
m.set_center(-71.12532, 42.3712, 12)
m.add_layer(mosaic, {}, 'spatial mosaic')
m

{'type': 'Image',
 'bands': [{'id': 'R',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 4294967295},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'G',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 4294967295},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 4294967295},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'N',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 4294967295},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

Map(center=[42.3712, -71.12532], controls=(WidgetControl(options=['position', 'transparent_bg'], position='top…